## Statistical Tests

In [1]:
import numpy as np
import os
from os.path import join as pjoin
from scipy.io import loadmat
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from viz import viz_df_trans
from scipy.stats import wilcoxon, chisquare

#### define functions

In [2]:
def sig_test(corr, ori_corr, resize, modality='meg'):

    # if resize=='ratio':
    #     stats_df = pd.DataFrame(index=list(corr.index), columns=list(corr.columns)[:-1])
    # else:
    stats_df = pd.DataFrame(index=list(corr.index), columns=list(corr.columns))

    for rescale, col in corr.iteritems():
        for layer_idx, scores in col.iteritems():
            # if not (rescale == 'rescale=1' and resize == 'ratio'):
            if modality == 'meg':
                d = np.mean(scores, axis=0) - np.mean(ori_corr[layer_idx], axis=0)
                if d.all() == 0:
                    stats_df[rescale][layer_idx] = np.nan
                else:
                    w,p = wilcoxon(np.mean(scores, axis=0), np.mean(ori_corr[layer_idx], axis=0))
                    if np.mean(d) > 0:
                        stats_df[rescale][layer_idx] = p
                    else:
                        stats_df[rescale][layer_idx] = -p

            elif modality == 'behavior':
                d = scores.flatten() - ori_corr[layer_idx].flatten()
                if d.all() == 0:
                    stats_df[rescale][layer_idx] = np.nan
                else:
                    w,p = wilcoxon(scores.flatten(), ori_corr[layer_idx].flatten())
                    if np.mean(d) > 0:
                        stats_df[rescale][layer_idx] = p
                    else:
                        stats_df[rescale][layer_idx] = -p

            elif modality == 'fmri':
                stats_dict = {'evc':stats_df, 'hvc':stats_df}
                for i, roi in enumerate(['evc', 'hvc']):
                    d = scores[:,i] - ori_corr[layer_idx][:,i]
                    if d.all() == 0:
                        stats_df[rescale][layer_idx] = np.nan
                    else:
                        w,p = wilcoxon(scores[:,i], ori_corr[layer_idx][:,i])
                        if np.mean(d) > 0:
                            stats_dict[roi][rescale][layer_idx] = p
                        else:
                            stats_dict[roi][rescale][layer_idx] = -p

    if modality == 'fmri':
        return stats_dict
    else:
        return stats_df

#### two-sided wilcoxon signed-rank test for each conditions

In [3]:
layer_list = np.arange(1, 6)
datasets_n = [92, 118]
modalities = ['meg', 'fmri', 'behavior']
resize_type = ['ratio', 'fixed']
dataset_root = '/LOCAL/ydai/workingdir/dataset'
results_root = '/LOCAL/ydai/workingdir/resizing_results'

In [4]:
# meg data
meg_sig_test = {}
for resize in resize_type:
    meg_sig_test[resize] = {}
    for set_idx in datasets_n:
    
        meg_dir = pjoin(results_root, 'meg')
        ori_corr = pd.read_pickle(pjoin(meg_dir, f'dataset{set_idx}_ratio_corr.pkl'))
        ori_corr = ori_corr.iloc[:,-1]

        corr_fn = f'dataset{set_idx}_{resize}_corr.pkl'
        corr = pd.read_pickle(pjoin(meg_dir, corr_fn))
        
        meg_sig_test[resize][f'dataset{set_idx}'] = sig_test(corr, ori_corr, resize, modality='meg')

        

In [5]:
# fmri data
fmri_sig_test = {}
for resize in resize_type:
    fmri_sig_test[resize] = {}
    for set_idx in datasets_n:
        fmri_dir = pjoin(results_root, 'fmri')
        ori_corr = pd.read_pickle(pjoin(fmri_dir, f'dataset{set_idx}_ratio_corr.pkl'))
        ori_corr = ori_corr.iloc[:,-1]

        corr_fn = f'dataset{set_idx}_{resize}_corr.pkl'
        corr = pd.read_pickle(pjoin(fmri_dir, corr_fn))

        fmri_sig_test[resize][f'dataset{set_idx}'] = sig_test(corr, ori_corr, resize, modality='fmri')
        

In [6]:
# behavior data
behavior_sig_test = {}
for resize in resize_type:
    behavior_sig_test[resize] = {}
    for set_idx in datasets_n:
        behavior_dir = pjoin(results_root, 'behavior')
        ori_corr = pd.read_pickle(pjoin(behavior_dir, f'dataset{set_idx}_ratio_corr.pkl'))
        ori_corr = ori_corr.iloc[:,-1]
        
        corr_fn = f'dataset{set_idx}_{resize}_corr.pkl'
        corr = pd.read_pickle(pjoin(behavior_dir, corr_fn))

        behavior_sig_test[resize][f'dataset{set_idx}'] = sig_test(corr, ori_corr, resize, modality='behavior')
        

#### Descriptive Statistical Results

In [58]:
# test all conditions
n_cond = 0
n_pos_sig = 0
for set_idx in datasets_n:
    meg_df = meg_sig_test['ratio'][f'dataset{set_idx}']
    fmri_df = fmri_sig_test['ratio'][f'dataset{set_idx}']
    behavior_df = behavior_sig_test['ratio'][f'dataset{set_idx}']

    n_cond += meg_df.size - meg_df.isna().sum().sum()
    n_cond += behavior_df.size - behavior_df.isna().sum().sum()
    n_cond += fmri_df['evc'].size - fmri_df['evc'].isna().sum().sum()
    n_cond += fmri_df['hvc'].size - fmri_df['hvc'].isna().sum().sum()

    n_pos_sig += meg_df[(meg_df>0) & (meg_df<0.05)].count().sum()
    n_pos_sig += behavior_df[(behavior_df>0) & (behavior_df<0.05)].count().sum()
    n_pos_sig += fmri_df['evc'][(fmri_df['evc']>0) & (fmri_df['evc']<0.05)].count().sum()
    n_pos_sig += fmri_df['hvc'][(fmri_df['hvc']>0) & (fmri_df['hvc']<0.05)].count().sum()

chisq, p = chisquare([n_pos_sig, n_cond-n_pos_sig])

print('{:.2f}% of downsampling cases lead to significant improvments, chi-square score={:.2f}, p={:.3f}'.format(n_pos_sig/n_cond*100, chisq, p))
    

61.00% of downsampling cases lead to significant improvments, chi-square score=9.68, p=0.002


In [48]:
# consider fmri data only
n_cond = 0
n_pos_sig = 0
for set_idx in datasets_n:
    fmri_df = fmri_sig_test['ratio'][f'dataset{set_idx}']

    n_cond += fmri_df['evc'].size - fmri_df['evc'].isna().sum().sum()
    n_cond += fmri_df['hvc'].size - fmri_df['hvc'].isna().sum().sum()

    n_pos_sig += fmri_df['evc'][fmri_df['evc']>0].count().sum()
    n_pos_sig += fmri_df['hvc'][fmri_df['hvc']>0].count().sum()

chisq, p = chisquare([n_pos_sig, n_cond-n_pos_sig])

print('{:.2f}% of downsampling cases of fmri data lead to improvments, chi-square score={:.2f}, p={:.3f}'.format(n_pos_sig/n_cond*100, chisq, p))
    

86.00% of downsampling cases of fmri data lead to improvments, chi-square score=51.84, p=0.000


In [71]:
# consider 25% downsampling only
n_cond = 0
n_pos_sig = 0
for set_idx in datasets_n:
    meg_df = meg_sig_test['ratio'][f'dataset{set_idx}']['rescale=0.25']
    fmri_df = fmri_sig_test['ratio'][f'dataset{set_idx}']
    behavior_df = behavior_sig_test['ratio'][f'dataset{set_idx}']['rescale=0.25']

    n_cond += meg_df.size - meg_df.isna().sum()
    n_cond += behavior_df.size - behavior_df.isna().sum()
    n_cond += fmri_df['evc']['rescale=0.25'].size - fmri_df['evc']['rescale=0.25'].isna().sum()
    n_cond += fmri_df['hvc']['rescale=0.25'].size - fmri_df['hvc']['rescale=0.25'].isna().sum()

    n_pos_sig += meg_df[meg_df>0].count().sum()
    n_pos_sig += behavior_df[behavior_df>0].count().sum()
    n_pos_sig += fmri_df['evc']['rescale=0.25'][(fmri_df['evc']['rescale=0.25']>0)].count().sum()
    n_pos_sig += fmri_df['hvc']['rescale=0.25'][(fmri_df['hvc']['rescale=0.25']>0)].count().sum()

chisq, p = chisquare([n_pos_sig, n_cond-n_pos_sig])

print('downsampling features to 25% causes improvments in {:.2f}% of cases, chi-square score={:.2f}, p={:.3f}'.format(n_pos_sig/n_cond*100, chisq, p))
    

downsampling features to 25% causes improvments in 85.00% of cases, chi-square score=19.60, p=0.000


In [72]:
# consider downsampling for layer1 only
n_cond = 0
n_pos_sig = 0
for set_idx in datasets_n:
    meg_df = meg_sig_test['ratio'][f'dataset{set_idx}'].iloc[0,:]
    fmri_df = fmri_sig_test['ratio'][f'dataset{set_idx}']
    behavior_df = behavior_sig_test['ratio'][f'dataset{set_idx}'].iloc[0,:]

    n_cond += meg_df.size - meg_df.isna().sum()
    n_cond += behavior_df.size - behavior_df.isna().sum()
    n_cond += fmri_df['evc'].iloc[0,:].size - fmri_df['evc'].iloc[0,:].isna().sum()
    n_cond += fmri_df['hvc'].iloc[0,:].size - fmri_df['hvc'].iloc[0,:].isna().sum()

    n_pos_sig += meg_df[meg_df>0].count().sum()
    n_pos_sig += behavior_df[behavior_df>0].count().sum()
    n_pos_sig += fmri_df['evc'].iloc[0,:][(fmri_df['evc'].iloc[0,:]>0)].count().sum()
    n_pos_sig += fmri_df['hvc'].iloc[0,:][(fmri_df['hvc'].iloc[0,:]>0)].count().sum()

chisq, p = chisquare([n_pos_sig, n_cond-n_pos_sig])

print('For features from layer1, downsampling leads improvments in {:.2f}% of cases, chi-square score={:.2f}, p={:.3f}'.format(n_pos_sig/n_cond*100, chisq, p))
    

For features from layer1, downsampling leads improvments in 80.00% of cases, chi-square score=14.40, p=0.000
